In [1]:
import pandas as pd

import os

import glob
import json

from textblob import TextBlob as tb

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

In [2]:
def read_json_file(file):
    with open(file, "r", encoding='utf8') as r:
        response = r.read()
        response = response.replace('\n', '')
        response = response.replace('}{', '},{')
        response = "[" + response + "]"
        return json.loads(response)

In [3]:
#função de diminuição de dimensionalidade de listas
def flatten_list(_2d_list):
    flat_list = []
    for element in _2d_list:
        if type(element) is list:
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

In [4]:
caminho_CL = r"DATA"
list_CL = glob.glob(os.path.join(caminho_CL, '*/*'))

# extraindo json e criando lista
dict_CL = list()
for i in range(len(list_CL)):
    dict_CL.append(read_json_file(list_CL[i]))
dict_CL = flatten_list(dict_CL)

In [5]:
#Extração do "place_id" para coleta de dados geográficos
data_extraction = []
for i in range(len(dict_CL)):
    try:
        for j in range(len(dict_CL[i]['data'])):
            place_id = dict_CL[i]['data'][j]['geo']['place_id']
            data_extraction.append(place_id) 
    except:
        continue  
data_extraction = list(set(data_extraction))

In [35]:
# # Bot de extração de dados geográficos
# # Para execução é necessário tirar os comentários do código
# login = input('Insira seu nome no twitter (tag)') #Colocar como input
# password = input('Insira sua senha') #Colocar como input
# options = webdriver.ChromeOptions() #Execução do webdriver do google
# options.add_experimental_option("prefs", {
#   "download.default_directory": r"E:\Blue\Bootcamp 2\github\bootcamp_covid\data_extraction\CHILE\GEO"
#   })
# browser = webdriver.Chrome(options=options)
# browser.get('https://twitter.com/')
# time.sleep(2)
# # Os elementos serão acessados via xpath, que é o path do HTML do site, buscando botões e caixa de interação
# browser.find_element('xpath', '/html/body/div[1]/div/div/div[1]/div/div[1]/div/div/div/div/div/div/div/div[1]/a/div/span/span').click()
# time.sleep(2)
# # É seguro inserir seus dados, visto que o algoritmo só guardará seus dados na memória do seu computador
# browser.find_element('xpath', '//html/body/div[1]/div/div/div[1]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div/div/div/div[5]/label/div/div[2]/div/input').send_keys(login)
# time.sleep(2)
# browser.find_element('xpath', '/html/body/div[1]/div/div/div[1]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div/div/div/div[6]/div').click()
# time.sleep(2)
# browser.find_element('xpath', '/html/body/div[1]/div/div/div[1]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[1]/div/div/div[3]/div/label/div/div[2]/div[1]/input').send_keys(password)
# time.sleep(2)
# browser.find_element('xpath', '/html/body/div[1]/div/div/div[1]/div[2]/div/div/div/div/div/div[2]/div[2]/div/div/div[2]/div[2]/div[2]/div/div[1]/div/div/div/div/span/span').click()
# time.sleep(2)
# # Um loop para acessar os dados do "place_id" que estão dentro da lista data_extraction
# for place_id in data_extraction:  
#   browser.get(f'https://api.twitter.com/1.1/geo/id/{place_id}.json')
# # A API do twitter restringe o uso da coleta de "place_id" em 100 vezes/hora então esse tempo é calculado
# # prevendo possíveis "bugs" e considerando o tempo limite por hora
#   time.sleep(40)

In [6]:
path = os.listdir('GEO')
dict_CL_GEO = list()
for i in path:
    with open(f'GEO/{i}', 'r', encoding='utf8') as f:
        dict_CL_GEO.append(json.load(f))

In [12]:
# A estrutura da lista funciona com o seguinte padrão:
# O primeiro objeto (dict_AR[i]) é o tweet de várias pessoas que falaram sobre o assunto covid naquele dia
# Dentro do objeto citado anteriormente podemos acessar as seguintes informações:
######### data #########
# Inclui os dados do tweet em si 
######### includes #########
# Inclui os dados do usuário que tweetou
######### meta #########
# Inclui informações sobre a pesquisa
# Com isso, as informações que serão coletadas serão:
# - Texto
# - Data
# - Quantidade de Retweet
# - Quantidade de likes
# - Grau inicial de polaridade (Utilizando a biblioteca TextBlob)
# O grau inicial será levado como consideração como baseline inicial

# all_text = []
# for i in range(len(dict_AR)):
#     try:
#         for j in range(len(dict_AR[i]['data'])):
#             text = dict_AR[i]['data'][j]['text']
#             for tweet in [dict_AR[i]['data'][j]['text']]:
#                 analysis = tb(tweet)
#                 polarity = analysis.sentiment.polarity
#             retweet = dict_AR[i]['data'][j]['public_metrics']['retweet_count']
#             like = dict_AR[i]['data'][j]['public_metrics']['like_count']
#             date = dict_AR[i]['data'][j]['created_at'].split('T')[0]
#             all_text.append(dict(text = text,
#                                 score = polarity,
#                                 retweet = retweet,
#                                 like = like,
#                                 date = date
#                                 )) 
#     except:
#         continue  

all_text = []
for i in range(len(dict_CL)):
    try:
        for j in range(len(dict_CL[i]['data'])):
            text = dict_CL[i]['data'][j]['text']
            for tweet in [dict_CL[i]['data'][j]['text']]:
                analysis = tb(tweet)
                polarity = analysis.sentiment.polarity
            retweet = dict_CL[i]['data'][j]['public_metrics']['retweet_count']
            like = dict_CL[i]['data'][j]['public_metrics']['like_count']
            date = dict_CL[i]['data'][j]['created_at'].split('T')[0]
            for z in range(len(dict_CL_GEO)):
                if dict_CL_GEO[z]['id'] == dict_CL[i]['data'][j]['geo']['place_id']:
                    lat = dict_CL_GEO[z]['centroid'][1]
                    long = dict_CL_GEO[z]['centroid'][0]
            all_text.append(dict(text = text,
                                score = polarity,
                                retweet = retweet,
                                like = like,
                                date = date,
                                lat = lat,
                                long = long
                                )) 
    except:
        continue  

In [14]:
CL_df = pd.DataFrame(all_text)
CL_df.to_csv('CL_data.csv')